In [1]:
import requests

In [2]:
URL="https://app-aiml24.cloud.sdu.dk/v1/completions"

***
***

In [3]:
class UCloudRequester:
    """ This class is a simple container for the UCloud API 
    request. It is used to send a query to the UCloud API 
    and return the response with or without metadata."""

    def __init__(self, url : str) -> None:
        self.url = url # URL of the UCloud API

    def call_url(self, prompt : str, **params) -> dict:
        """This method handles the call to the UCloud API and
        returns the response as a dictionary"""

        # try:
        query = {'prompt': prompt, **params}
        response = requests.post(
            url=self.url,
            headers={'Content-Type': 'application/json'},
            json=query
        )
        return response.json() # <- Translates JSON to dictionary
        
        # except requests.exceptions.JSONDecodeError:
        #     return f"JSONDecodeError: Failed to decode response as JSON: {response.text}"

        # except requests.exceptions.HTTPError as e:
        #     return "HTTPError: " + str(e)
        
    def predict(self, prompt : str, return_meta=False, **params) -> dict:
        """This method is the main method for sending a request.
        if return_meta is set to True, the method will return the
        full response as a dictionary. Otherwise, it will return
        the text of the first choice in the response."""

        result : dict = self.call_url(prompt, **params)
        if return_meta:
            return result
        return result["choices"][0]["text"]
    

requester = UCloudRequester(
    url=URL,
)

***
***

In [4]:
# Example model parameters
params = dict(
    max_tokens=50,
    temperature=0.7,
    stop=[],
)

In [5]:
PROMPT = """Write the first verse of the most famous Danish song: \nVerse:""" 

In [6]:
PROMPT="did masks help stop the spread of the virus?"

In [16]:
prediction = requester.predict(prompt=PROMPT, return_meta=True, **params)

In [17]:
prediction

{'id': 'cmpl-559a55ea-f72e-40e1-8f68-9bce1af54a1f',
 'object': 'text_completion',
 'created': 1710465414,
 'model': 'models/mixtral-8x7b-instruct-v0.1.Q5_K_M.gguf',
 'choices': [{'text': "Infinity, a concept vast and wide,\nBeyond our reach, yet in our minds it resides.\nA symbol, eight, turned on its side,\nRepresents eternal life, with nowhere to hide.\n\nIt'",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 16, 'completion_tokens': 50, 'total_tokens': 66}}

In [9]:
PROMPT = """Write a short poem about the meaning of life.
Poem:

"""

In [10]:
prediction = requester.predict(prompt=PROMPT, return_meta=True, **params)

In [11]:
print(prediction["choices"][0]["text"])

In this vast universe, we spin and twirl,
Our atoms merging with the world.
A dance of energy, so wild and free,
Yet bound to earth, as we must be.

Each moment precious, not


In [12]:
PROMPT = """Write a {} about the meaning of {}.
Poem:

"""

In [92]:
%%timeit -n 3 -r 3

for fmt, xtra in zip(["life", "love", "happiness", "sadness", "freedom"], ["short story", "poem", "testament", "song", "ode"]):
    prediction = requester.predict(prompt=PROMPT.format(fmt, xtra), return_meta=True, **params)

5.1 s ± 242 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [68]:
%%timeit -n 10 -r 10

requester.predict(f"The meaning of life is", **params)

741 ms ± 134 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
